In [0]:
%run "/Workspace/Users/rehneet11_outlook.com#ext#@rehneet11outlook.onmicrosoft.com/course/Formula1/includes/common_functions"

## Ingest circuits.csv File

In [0]:
dbutils.widgets.text("p_data_source", "")
v_data_source=dbutils.widgets.get("p_data_source")

In [0]:
dbutils.widgets.text("p_file_date", "")
v_file_date=dbutils.widgets.get("p_file_date")

#### Applying Custom Schema

In [0]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType
circuit_schema=StructType(fields=[StructField("circuitId",IntegerType(),False),
                                  StructField("circuitRef",StringType(),True),StructField("name",StringType(),True),StructField("location",StringType(),True),StructField("country",StringType(),True),StructField("lat",DoubleType(),True),StructField("lng",DoubleType(),True),StructField("alt",IntegerType(),True),StructField("url",StringType(),True)
                                  ]
                          )


In [0]:
df=spark.read.options(header=True).schema(circuit_schema).csv(f"{raw_folder_path}/{v_file_date}/circuits.csv")
df.printSchema()

#### Selecting columns

In [0]:
from pyspark.sql.functions import col,lit
selected_df=df.select(col("circuitId"),col("circuitRef"),col("name"),col("location"),col("country"),col("lat"),col("lng"),col("alt"))
selected_df.display()

#### Renaming the column

In [0]:
renamed_df=selected_df.withColumnRenamed("circuitId","circuit_id")\
.withColumnRenamed("circuitRef","circuit_ref")\
.withColumnRenamed("lat","latitude")\
.withColumnRenamed("lng","longitude")\
.withColumnRenamed("alt","altitude")\
.withColumn("data_source",lit(v_data_source))\
.withColumn("file_date",lit(v_file_date))
renamed_df.display()

#### Add Ingestion Date as a new Column

In [0]:
final_df=add_ingestion_date(renamed_df)

#### Write Data in DL using parquet format

In [0]:
final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_processed.circuits")